In [1]:
import pandas as pd
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import folium

In [2]:
earliest_arrival_times = pd.read_csv('./data/raptor_results.csv')

In [3]:
stops_df = pd.read_csv('./data/cleaned/stops.csv')
stops_gdf = gpd.GeoDataFrame(stops_df, geometry=gpd.points_from_xy(stops_df.stop_lon, stops_df.stop_lat))

In [4]:
earliest_arrival_times.sort_values(by=['arrival_time'])
earliest_arrival_times = earliest_arrival_times.set_index('stop_id')
earliest_arrival_times["is_reached"] = earliest_arrival_times["arrival_time"] != "--:--:--"
earliest_arrival_times

,arrival_time,is_reached
stop_id,,
1333153,15:59:03,True
1282147,16:06:39,True
1122116,15:42:56,True
1230104,--:--:--,False
1180214,15:37:02,True
...,...,...
1020119,15:26:22,True
1285159,16:05:47,True
1240419,15:46:53,True


In [5]:
sample_stop = stops_gdf.sample(1)
m = folium.Map(location=[sample_stop.stop_lat, sample_stop.stop_lon], zoom_start=12)
for i, row in stops_gdf.iterrows():
    eat = earliest_arrival_times.loc[row.stop_id]
    is_reached = eat["is_reached"]
    color = "green" if is_reached else "red"
    folium.CircleMarker(
        location=[row.stop_lat, row.stop_lon],
        radius=1,
        color=color,
        popup=f"{row.stop_name} ({row.stop_id}) @ {eat['arrival_time']}",
    ).add_to(m)
m


/home/moritz/miniconda3/envs/mcr-py/lib/python3.10/site-packages/folium/utilities.py:69: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(coord)
/home/moritz/miniconda3/envs/mcr-py/lib/python3.10/site-packages/folium/utilities.py:77: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  if math.isnan(float(coord)):
/home/moritz/miniconda3/envs/mcr-py/lib/python3.10/site-packages/folium/utilities.py:79: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return [float(x) for x in coords]
